# S57 Utils

# S-57 Reference Data and Utilities

This notebook demonstrates the `S57Utils` class, which provides access to S-57 reference data:
- **Object class definitions**: 217 object types defined in S-57 standard
- **Attribute definitions**: 304 standard attributes for objects
- **Property enumerations**: 1400+ coded values for attributes

This reference data is essential for:
- Understanding what data is contained in your ENC database
- Converting coded values to human-readable meanings
- Data validation and quality assurance
- Building maritime analysis applications

## S-57 Data Structure
S-57 files contain three levels of organization:

- **Objects** (layers): Features like soundings (SOUNDG), buoys (BOYSAN), lights (LIGHTS)
- **Attributes**: Properties of objects (e.g., color, depth, status)
- **Properties**: Coded values for attributes (e.g., color=1 means "red")

This notebook shows how to work with all three levels.

## Data Sources

- **Object/Attribute Reference**: IHO S-57 standard definitions (bundled with toolkit)
- **NOAA Chart Database**: Live web scraping from official NOAA catalog (requires internet connection)
- **Property Enumerations**: Code-to-meaning mappings from S-57 specification

## 1. S-57 Reference Data: Objects, Attributes, and Properties

Access the complete S-57 object and attribute catalog.
This section retrieves reference tables used throughout the nautical_graph_toolkit.


In [ ]:
# =============================================================================
# SETUP AND IMPORTS
# =============================================================================

# --- Standard Library ---
import sys
import os
from pathlib import Path

# --- Environment Setup ---
from dotenv import load_dotenv

# --- Project Setup ---
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Load environment variables
load_dotenv(project_root / '.env')

# --- Project Imports ---
from src.nautical_graph_toolkit.utils.s57_utils import S57Utils, NoaaDatabase
from src.nautical_graph_toolkit.core.s57_data import ENCDataFactory, S57AdvancedConfig, S57Advanced

# --- Initialize S57Utils ---
utils = S57Utils()

print("=" * 70)
print("✓ All imports loaded successfully")
print("=" * 70)

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# --- Database Backend (for examples) ---
db_schema = "enc_west"  # Change to your schema name

# --- Layer Examples ---
example_layer = "boyspp"  # Layer to demonstrate conversions (buoys in special position)

# --- File-Based Example ---
example_gpkg = "enc_west.gpkg"  # GeoPackage file for file-based examples

# =============================================================================
# CONFIGURATION SUMMARY
# =============================================================================

print("=" * 70)
print("✓ Configuration loaded")
print("=" * 70)
print(f"Database schema: {db_schema}")
print(f"Example layer: {example_layer}")
print(f"Example file: {example_gpkg}")
print("=" * 70)

### S-57 Object Classes (Layers)

S-57 defines 217 object classes representing different types of maritime features.
Each object has:
- **Code**: Numeric identifier
- **Name**: Full English description
- **Acronym**: 6-letter code (e.g., 'soundg' = sounding)
- **Attributes**: Which properties can apply to this object
- **Primitives**: Geometry types (point, line, area)

The table below shows the first few:


In [ ]:
utils.get_objects_df()

### S-57 Attributes

Attributes are properties that can be attached to S-57 objects.
The reference table shows:
- **Code**: Numeric attribute identifier
- **Attribute**: Full name
- **Type**: Data type (enumerated, integer, float, etc.)
- **Class**: Object class that uses this attribute

For example, 'BCNSHP' (beacon shape) is enumerated with values like 1=stake, 2=tower, etc.


In [ ]:
utils.get_attributes_df()

### S-57 Property Values (Enumerations)

For enumerated attributes, codes map to meanings:
- **Code**: The numeric value stored in the ENC
- **Attribute**: Which attribute this code belongs to
- **Meaning**: Human-readable value (e.g., 1 = 'stake, pole, perch, post')

This allows conversion from cryptic numbers to understandable labels.


In [ ]:
utils.get_properties_df()

## 2. Practical Example: Converting Acronyms and Properties

Demonstrate how to load ENC data and apply S-57 conversions for analysis.

### Conversion Options
- **convert_acronyms=True**: Replace 6-letter codes with full English names
- **convert_properties=True**: Replace enumerated codes with their meanings
- **prop_mixed=True**: Keep both codes and meanings in output

This makes ENC data human-readable and suitable for reporting and analysis.


Tests the new conversion features in ENCDataFactory.get_layer.

### PostGIS

In [ ]:
db_params = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

# Validate database parameters
required_params = ['dbname', 'user', 'password', 'host', 'port']
missing_params = [p for p in required_params if not db_params[p]]

if missing_params:
    print(f"❌ Missing database parameters: {', '.join(missing_params)}")
    print("   Please configure your .env file with database credentials")
    raise ValueError("Cannot proceed without database configuration")

print("\n--- Running test: Importing Dataset ---")
try:
    pg_factory = ENCDataFactory(source=db_params, schema=db_schema)
    print(f"✓ Connected to database '{db_params['dbname']}'")
except Exception as e:
    print(f"❌ Failed to connect to database: {e}")
    print("   Please verify your database credentials and connection")
    raise

print("\n--- Running test: Acronym and Property Conversion ---")
layer_name = example_layer

# 1. Get raw data
try:
    gdf_raw = pg_factory.get_layer(layer_name)
    print(f"    ✅ Fetched raw data successfully ({len(gdf_raw)} features).")
except Exception as e:
    print(f"    ❌ Failed to fetch layer '{layer_name}': {e}")
    raise

gdf_raw

#### 2. Get data with converted acronyms

In [ ]:
# 2. Get data with converted acronyms
gdf_acronyms = pg_factory.get_layer(layer_name, convert_acronyms=True, prop_mixed=False)

print("    ✅ Converted acronyms successfully.")
gdf_acronyms

#### 3. Get data with converted properties

In [ ]:
# 3. Get data with converted properties
gdf_props = pg_factory.get_layer(layer_name, convert_acronyms=True, convert_properties=True, prop_mixed=True)

# Verify property conversion succeeded
if 'Buoy shape (boyspp)' in gdf_props.columns or any('shape' in str(c).lower() for c in gdf_props.columns):
    print("    ✅ Converted properties successfully.")
else:
    print("    ⚠️  Property conversion completed but attribute not found in output")

gdf_props

### File-Based DB

In [ ]:
file_path = Path.cwd() / 'output' / example_gpkg

# Validate file exists
if not file_path.exists():
    print(f"❌ File not found: {file_path}")
    print("   Please ensure the GeoPackage file has been created via import_s57.ipynb")
    raise FileNotFoundError(f"Cannot proceed without {file_path}")

print("\n--- Running test: Importing Dataset ---")
try:
    file_factory = ENCDataFactory(source=file_path)
    print(f"✓ Connected to GeoPackage '{example_gpkg}'")
except Exception as e:
    print(f"❌ Failed to open GeoPackage: {e}")
    raise

print("\n--- Running test: Acronym and Property Conversion ---")
layer_name = example_layer

# 1. Get raw data
try:
    gdf_raw = file_factory.get_layer(layer_name)
    print(f"    ✅ Fetched raw data successfully ({len(gdf_raw)} features).")
except Exception as e:
    print(f"    ❌ Failed to fetch layer '{layer_name}': {e}")
    raise

gdf_raw

#### 2. Get data with converted acronyms

In [ ]:
gdf_acronyms = file_factory.get_layer(layer_name, convert_acronyms=True, prop_mixed=False)

print("    ✅ Converted acronyms successfully.")
gdf_acronyms

#### 3. Get data with converted properties

In [ ]:
gdf_props = file_factory.get_layer(layer_name, convert_acronyms=True, convert_properties=True, prop_mixed=True)

print("    ✅ Converted properties successfully.")
gdf_props

## 3. NOAA Chart Database

Access the live NOAA ENC database to check for latest chart versions.

This utility:
- Scrapes the official NOAA chart catalog
- Returns DataFrame with latest edition and update numbers
- Enables validation of your local data against official versions
- Requires internet connection

Useful for maintenance workflows: identify outdated charts and plan updates.


In [ ]:
print("\n--- Running test: NOAA ENC Database ---")
try:
    noaa_db = NoaaDatabase()
    noaa_charts = noaa_db.get_charts(force_refresh=True, as_dataframe=True)
    
    if noaa_charts is None or noaa_charts.empty:
        print("⚠️  No chart data retrieved from NOAA")
    else:
        print(f"✓ Successfully retrieved {len(noaa_charts)} charts from NOAA database")
        print(f"  Latest update: {noaa_charts['issue_date'].max()}")
    
    display(noaa_charts)
    
except Exception as e:
    print(f"❌ Failed to fetch NOAA data: {e}")
    print("   This typically indicates:")
    print("   - No internet connection")
    print("   - NOAA website is unavailable")
    print("   - Network firewall/proxy issues")
    print("   Attempting to use cached data if available...")
    try:
        noaa_db = NoaaDatabase()
        noaa_charts = noaa_db.get_charts(as_dataframe=True)
        if noaa_charts is not None:
            print(f"✓ Using cached data ({len(noaa_charts)} charts)")
        else:
            print("⚠️  No cached data available")
    except:
        print("⚠️  Unable to retrieve NOAA data (online or cached)")